# Validation of mapped area dataset

In [11]:
import polars as pl
pl.enable_string_cache()

In [16]:
df = pl.scan_parquet("data/uk_areas_lookup.parquet").lazy()
df.describe()

statistic,OA21CD,LSOA21CD,LSOA21NM,LSOA11CD,CHGIND,MSOA21CD,MSOA21NM,WD24CD,WD24NM,LAD22CD,LAD22NM
str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""189944""","""189944""","""189944""","""189944""","""189944""","""189944""","""189944""","""189944""","""189944""","""189944""","""189944"""
"""null_count""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0"""
"""mean""",null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null
"""min""",null,null,"""Adur 001A""",null,null,null,"""Adur 001""",null,"""Abbey""",null,"""Adur"""
"""25%""",null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,null,null,null,null,null,null,null,null,null,null
"""max""",null,null,"""York 024F""",null,null,null,"""York 024""",null,"""Zetland""",null,"""York"""


In [17]:
summary = (
    df.select([
        pl.col("OA21CD").n_unique().alias("OA_2021"),
        pl.col("LSOA21CD").n_unique().alias("LSOA_2021"),
        pl.col("MSOA21CD").n_unique().alias("MSOA_2021"),
        pl.col("WD24CD").n_unique().alias("Ward_2024"),
        pl.col("LAD22CD").n_unique().alias("LAD_2022"),
        pl.when(pl.col("CHGIND") == "U")
          .then(pl.col("LSOA21CD"))
          .n_unique()
          .alias("LSOA_unchanged")
    ])
    .collect()
)

oa_count = summary[0, 0]
lsoa_count = summary[0, 1]
msoa_count = summary[0, 2]
ward_count = summary[0, 3]
lad_count = summary[0, 4]
unchanged_lsoa_count = summary[0, 5]

print(f"OA2021       : {oa_count:,}")
print(f"LSOA2021     : {lsoa_count:,}")
print(f"MSOA2021     : {msoa_count:,}")
print(f"Ward2024     : {ward_count:,}")
print(f"LAD2022      : {lad_count:,}")
print(f"Unchanged LSOA: {unchanged_lsoa_count:,}")

OA2021       : 188,880
LSOA2021     : 35,672
MSOA2021     : 7,264
Ward2024     : 7,536
LAD2022      : 331
Unchanged LSOA: 33,648


In [18]:
london_summary = (
    df.filter(
        (pl.col("LAD22CD").cast(pl.Utf8).str.starts_with("E09")) |
        (pl.col("LAD22NM").cast(pl.Utf8).str.contains("London"))
    )
    .select([
        pl.col("OA21CD").n_unique().alias("London_OA_2021"),
        pl.col("LSOA21CD").n_unique().alias("London_LSOA_2021"),
        pl.col("MSOA21CD").n_unique().alias("London_MSOA_2021"),
        pl.col("WD24CD").n_unique().alias("London_Ward_2024"),
        pl.when(pl.col("CHGIND") == "U")
          .then(pl.col("LSOA21CD"))
          .n_unique()
          .alias("London_LSOA_unchanged")
    ])
    .collect()
)

# Extract the London counts
london_oa_count = london_summary[0, 0]
london_lsoa_count = london_summary[0, 1]
london_msoa_count = london_summary[0, 2]
london_ward_count = london_summary[0, 3]
london_unchanged_lsoa_count = london_summary[0, 4]

# Print the existing counts
print("United Kingdom Statistics:")
print(f"OA2021       : {oa_count:,}")
print(f"LSOA2021     : {lsoa_count:,}")
print(f"MSOA2021     : {msoa_count:,}")
print(f"Ward2024     : {ward_count:,}")
print(f"LAD2022      : {lad_count:,}")
print(f"Unchanged LSOA: {unchanged_lsoa_count:,}")

# Print the London-specific counts
print("\nLondon Statistics:")
print(f"London OA2021       : {london_oa_count:,}")
print(f"London LSOA2021     : {london_lsoa_count:,}")
print(f"London MSOA2021     : {london_msoa_count:,}")
print(f"London Ward2024     : {london_ward_count:,}")
print(f"London Unchanged LSOA: {london_unchanged_lsoa_count:,}")

# Calculate and print percentages
print("\nLondon Percentages:")
print(f"London OAs        : {london_oa_count/oa_count:.1%} of all OAs")
print(f"London LSOAs      : {london_lsoa_count/lsoa_count:.1%} of all LSOAs")
print(f"London MSOAs      : {london_msoa_count/msoa_count:.1%} of all MSOAs")
print(f"London Wards      : {london_ward_count/ward_count:.1%} of all Wards")

United Kingdom Statistics:
OA2021       : 188,880
LSOA2021     : 35,672
MSOA2021     : 7,264
Ward2024     : 7,536
LAD2022      : 331
Unchanged LSOA: 33,648

London Statistics:
London OA2021       : 26,369
London LSOA2021     : 4,994
London MSOA2021     : 1,002
London Ward2024     : 684
London Unchanged LSOA: 4,660

London Percentages:
London OAs        : 14.0% of all OAs
London LSOAs      : 14.0% of all LSOAs
London MSOAs      : 13.8% of all MSOAs
London Wards      : 9.1% of all Wards


In [19]:
london = pl.scan_parquet("data/london_areas_lookup.parquet").lazy()
london.describe()

statistic,OA21CD,LSOA21CD,LSOA21NM,LSOA11CD,CHGIND,MSOA21CD,MSOA21NM,WD24CD,WD24NM,LAD22CD,LAD22NM
str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""26559""","""26559""","""26559""","""26559""","""26559""","""26559""","""26559""","""26559""","""26559""","""26559""","""26559"""
"""null_count""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0"""
"""mean""",null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null
"""min""",null,null,"""Barking and Dagenham 001A""",null,null,null,"""Barking and Dagenham 001""",null,"""Abbey""",null,"""Barking and Dagenham"""
"""25%""",null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,null,null,null,null,null,null,null,null,null,null
"""max""",null,null,"""Westminster 024G""",null,null,null,"""Westminster 024""",null,"""Yiewsley""",null,"""Westminster"""


In [21]:
l_summary = (
    london.select([
        pl.col("OA21CD").n_unique().alias("OA_2021"),
        pl.col("LSOA21CD").n_unique().alias("LSOA_2021"),
        pl.col("MSOA21CD").n_unique().alias("MSOA_2021"),
        pl.col("WD24CD").n_unique().alias("Ward_2024"),
        pl.col("LAD22CD").n_unique().alias("LAD_2022"),
        pl.when(pl.col("CHGIND") == "U")
          .then(pl.col("LSOA21CD"))
          .n_unique()
          .alias("LSOA_unchanged")
    ])
    .collect()
)

oa_count = l_summary[0, 0]
lsoa_count = l_summary[0, 1]
msoa_count = l_summary[0, 2]
ward_count = l_summary[0, 3]
lad_count = l_summary[0, 4]
unchanged_lsoa_count = l_summary[0, 5]

print(f"OA2021       : {oa_count:,}")
print(f"LSOA2021     : {lsoa_count:,}")
print(f"MSOA2021     : {msoa_count:,}")
print(f"Ward2024     : {ward_count:,}")
print(f"LAD2022      : {lad_count:,}")
print(f"Unchanged LSOA: {unchanged_lsoa_count:,}")

OA2021       : 26,369
LSOA2021     : 4,994
MSOA2021     : 1,002
Ward2024     : 684
LAD2022      : 33
Unchanged LSOA: 4,660
